# AutoML 机制简介

AutoML 机制是 AlphaMed 平台提供的一套自动化训练、部署的解决方案。AutoML 机制的目标用户是在某个领域内有分析研究数据需求、使用人工智能提升业务需求、但又不具备代码编写能力的研究人员。通过在 AlphaMed 平台上创建数据集、标注训练数据，并选择合适的 AutoModel 预训练模型，可以在不编写一行代码的情况下获得针对自身数据特征微调后的模型，并完成部署助力实际业务。

# AutoModel 简介

AutoModel 是 AlphaMed 平台上用以支持 AutoML 机制的预训练模型，与 Huggingface、PyTorch、Paddle 等平台提供的预训练模型有明显不同。其它平台提供的预训练模型是以模型本身为核心而设计的，而 AlphaMed 平台的预训练模型以具体业务场景为核心。以 ResNet 预训练模型为例，在其它平台上，只有一套标准的 ResNet 预训练模型，一套对应的预训练参数。其预训练参数大多是基于论文中提到的公共数据集训练得到的，与具体使用场景无关。而 AlphaMed 平台上的 ResNet 预训练模型，可能被用于多个具体业务场景中，比如皮肤病诊断分类、眼底膜疾病预测、肺结节诊断等，且每个业务都有一套基于业务数据集训练得到的预训练参数，只是共享同一套 ResNet 网络结构。此外，对于某一个具体的业务，其使用的预训练模型也是可以发生变化的，比如皮肤病诊断分类的第一个版本可能使用 ResNet，而第二个版本可能改用 ViT 以获得更好的效果。

AutoModel 主要完成两个核心任务：
1. 自动微调。传统预训练模型只提供模型主干，fine-tune 时需要算法工程师根据具体任务完成调整网络结构、编写运行脚本等编码工作，然后才能实际运行。AutoML 机制不依赖用户编程，所以需要 AutoModel 自身能够直接启动 fine-tune 流程。当然会因此引入一些限制，比如必须使用 AlphaMed 平台数据集框架支持，联邦算法模型必须在 AlphaMed 平台环境中训练等。
2. 自动推理。传统预训练模型推理时，需要算法工程师编写代码处理原始数据输入。AutoML 机制不依赖用户编程，所以需要 AutoModel 自身能够直接适配原始格式的数据输入，实际兼容能力依赖于模型代码本身的具体处理逻辑。

AutoModel 为此设计了一系列工具，以支持实现上述两个目标，同时尽可能在简单性、兼容性、灵活性之间取得平衡。

## AutoModel 机制

要完成自动微调和推理，核心是处理三件事：加载启动模型、加载转换原始数据、控制微调训练和推理流程。

后面结合一个具体示例，详细介绍这三个需求的实现机制。示例基于经典的 ResNet18 网络，加载预训练参数后，针对 [HAM10000 数据集](https://alphamed-share-1309103037.cos.ap-beijing.myqcloud.com/HAM10000.zip)进行微调。

### 加载启动模型机制简介

加载启动模型功能通过 Model Zoo 配合模型配置文件机制实现。Model Zoo 负责提供加载模型的数据源，比如定义模型的 .py 文件或模块文件、模型参数文件、模型配置文件等。AutoModel 模块获得模型数据源后，根据事先确定好的配置文件机制，读取启动模型需要的核心参数，就可以将模型启动起来。

当前版本只支持源代码形式的模型定义，Script 类型的模型可能在未来支持。[ResNet 模型的代码](res/auto_model_local/res_net.py)没有什么特殊之处，在此不赘述。需要注意的是配套的 [config.json](res/auto_model_local/config.json) 文件。
*（resnet18.pth 文件可在[这里](https://alphamed-share-1309103037.cos.ap-beijing.myqcloud.com/resnet18.pth)下载。）*

```JSON
{
    "entry_file": "auto.py",
    "entry_module": null,
    "entry_class": "AutoResNet",
    "param_file": "resnet18.pth",
    "id2label": {
        "0": "AKIEC",
        "1": "BCC",
        "2": "BKL",
        "3": "DF",
        "4": "MEL",
        "5": "NV",
        "6": "VASC"
    },
    "label2id": {
        "AKIEC": 0,
        "BCC": 1,
        "BKL": 2,
        "DF": 3,
        "MEL": 4,
        "NV": 5,
        "VASC": 6
    },
    "learning_rate": 1e-4,
    "epochs": 20,
    "batch_size": 64
}
```

其中 `entry_file`、`entry_module`、`entry_class`、`param_file` 是所有 AutoModel 都必须配置的参数。`entry_file`、`entry_module` 配置了模型代码的定义文件（模块），如果模型是在一个 .py 文件中定义的，使用 `entry_file` 配置这个 .py 文件；如果模型是在一个 Python 模块（包含 \_\_init__.py 的文件夹）中定义的，使用 `entry_module` 配置这个模块文件夹。`entry_class` 用于配置 AutoModel 实现类，注意这里配置的是 AutoModel 实现类，而不是模型本身的实现类，比如示例中是 `AutoResNet` 而不是 `ResNet18`。`param_file` 用于配置模型对应的预训练参数文件。有了这四个配置，平台就能够读取代码源文件，从中找到模型类，据此实例化模型对象并加载预训练参数。

`id2label`、`label2id` 定义了标签类别序号和名称之间的映射关系。AlphaMed 平台定义配置项时，会尽量遵从业界最佳实践。

`learning_rate`、`epochs`、`batch_size` 是预训练中使用的超参数，也可以根据实现业务需要自由添加其它参数。

准备好了 AutoModel 的配置文件以及配置中指定的所有文件资源后，将它们统一放在一个目录下，示例中所有文件均放置在 res/auto_model_local/ 目录下。然后就可以通过平台的 `from_pretrained` 接口加载 AutoModel 模型实例了。

In [ ]:
from alphafed.auto_ml import from_pretrained

auto_model = from_pretrained(resource_dir='res/auto_model_local/')

如果配置、代码都能正常工作，`from_pretrained` 将返回一个可用的 AutoModel 对象。

### 加载转换原始数据机制简介

加载转换原始数据功能通过 AlphaMed 平台的数据集框架和预处理机制来实现。数据集框架提供相对统一的数据和标注格式，借助 AutoModel 中附带的针对性的预处理器，可以实现加载原始数据和标注，并转化为适配模型的输入格式。针对不同的原始数据格式和不同的模型，预处理器的设计使用方式也会存在差异。

原始数据类型可能是图片、文本等各种格式，在传入模型处理之前需要先转化为张量数据。而且在训练的过程中，可能还需要对原始数据做一些增强，比如随机旋转、裁切等。具体的处理方式和模型本身是高度相关的，因此可以针对每个模型配套提供一些 `Preprocessor` 工具。具体设计 AutoModel 预训练模型时，设计者要根据原始数据的格式和模型期望的输入格式，设计 `Preprocessor` 工具完成期望的数据转换。同时为了提高 `Preprocessor` 工具的使用灵活性，还可以通过 [`preprocessor_config.json`](res/auto_model_local/preprocessor_config.json) 文件动态调整预处理工序。

除了处理数据本身之外，还需要处理训练数据对应的标签信息。标签信息需要符合 AlphaMed 平台[数据标注规范](https://quizzdaily.feishu.cn/docx/Uh39daDG8otsOFxKEcLcpHHnnvc)。在平台上微调训练时，用户在平台上标注的数据标签会自动转换为符合规范的格式，通过 annotation.json 文件提供给 AutoModel 使用。所以 AutoModel 需要能够按照标注规范读取标注信息，否则不能在平台上运行。具体到示例的 HAM10000 图片分类任务，标注格式如下：

```JSON
{
    "labels": ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"],  // 标签列表
    "Train": [  // 训练集数据
        {  // 一个样本的数据
            "path": "ISIC_0032779.jpg",  // 图像文件路径（数据文件夹下的相对路径）
            "tags": [  // 用于图像分类的标注，没有标注的时候为 null 或空数组
                { "label": "NV", "source": "manual" }
            ]
        }, {  // 另一个样本的数据
            "path": "ISIC_0031743.jpg",  // 图像文件路径（数据文件夹下的相对路径）
            "tags": [  // 用于图像分类的标注，没有标注的时候为 null 或空数组
                { "label": "AKIEC", "source": "manual" }
            ]
        }, {
            // 另一个样本的数据
        }
    ],
    "Validation": [  // 验证集数据，没有数据的时候为 null 或空数组
        // 同 Train
    ],
    "Test": [  // 测试集数据，没有数据的时候为 null 或空数组
        // 同 Train
    ]
}
```

平台提供了 `ImageAnnotationUtils` 工具帮助处理数据和标签的加载，AutoModel 设计者也可以根据规范格式自行实现。实际训练时任务管理器会负责将训练所需的数据及标签下载到本地文件夹下，并在启动任务时告知文件夹地址，AutoModel 可以在文件夹中找到原始数据和标注文件。

### 控制微调训练流程机制简介

对于本地运行的 AutoModel，由于不涉及不同节点间的通信，开发者可以根据自己的喜好随意实现，在必要的时候通知任务管理器当前状态即可（当前任务管理器还不支持本地模式 AutoML，所以暂时未做集成）。对于联邦模式的 AutoModel，需要符合联邦算法调度器的要求，可参考之前几节的相关说明。这部分内容后续介绍联邦模式 AutoModel 时再详细说明。

介绍完了三个核心机制，现在开始逐步实现示例代码。

### 设计实现本地运行的 AutoModel

AutoModel 的加载是由平台自动处理的。对于开发者来讲，就是确保相关配置文件中的配置正确无误、相关资源文件不要缺失、以及排除代码中的 bug 即可，除此之外没有其它特别的要求。如前所述，任务管理器会负责将原始数据下载保存在指定的文件目录中，并告知 AutoModel 这个文件目录的位置。所以 AutoModel 只需要关注如何定义预处理器将数据转换为张量，以及如果读取解析 annotation.json 文件中的标注信息。

先说如何设计预处理器。预处理器必须继承 `Preprocessor` 基础类，实现其中的 `transform` 接口。这个接口接收原始图片的文件地址，经转化后返回模型可以处理的张量。

In [ ]:
from abc import ABC, abstractmethod
from typing import Any
import torch

class Preprocessor(ABC):

    @abstractmethod
    def transform(self, raw_input: Any, *args: Any, **kwargs: Any) -> torch.Tensor:
        """将原始输入转化为张量。"""

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        """与 transform 配合实现 processor(input) 形式的调用。"""
        return self.transform(*args, **kwargs)

示例中的预处理器实现接受图片文件地址，加载图片内容，并将其转换为张量。在转换图片的时候，可以指定预处理器的工作模式，如果是训练模式则在转换时会自动提供数据增强的功能。数据增强方式由 preprocessor_config.json 文件中的配置定义。注意这个设计方式不是 AutoModel 机制本身强制要求的，只是示例采用的一种方式。

In [ ]:
from PIL import Image
from torchvision import transforms

from alphafed.auto_ml.auto_model import DatasetMode
from res.auto_model_local.auto import PreprocessorConfig

class ResNetPreprocessor(Preprocessor):

    def __init__(self, mode: DatasetMode, config: PreprocessorConfig) -> None:
        layers = []
        layers.append(transforms.Resize((config.size, config.size)))
        if mode == DatasetMode.TRAINING:
            layers.append(transforms.RandomAffine(degrees=config.degrees,
                                                  translate=config.translate))
            layers.append(transforms.RandomHorizontalFlip())
        layers.append(transforms.ToTensor())
        layers.append(transforms.Normalize(config.image_mean, config.image_std))
        self._transformer = transforms.Compose(layers)

    def transform(self, image_file: str) -> torch.Tensor:
        """Transform an image object into an input tensor."""
        image = Image.open(image_file).convert('RGB')
        return self._transformer(image)

除了处理原始数据，还需要处理标注信息。任务运行时任务管理器会将数据集对应的标注信息保存在 annotation.json 文件中，放置在数据文件夹根目录下。开发者可以参考[标注格式规范](https://quizzdaily.feishu.cn/docx/Uh39daDG8otsOFxKEcLcpHHnnvc)自行开发解析工具，也可以直接使用 AlphaMed 提供的标注处理工具。目前平台提供的标注处理工具只有针对图片数据的 `ImageAnnotationUtils`，将来会根据平台支持的数据类型不断添加。

本示例中直接使用 `ImageAnnotationUtils` 工具解析标注信息。配合 `ResNetPreprocessor` 预处理器和 `ImageAnnotationUtils` 工具，完成设计了一个 `ResNetDataset` 实现，帮助 AutoModel 在训练、验证、测试的过程中更方便的批量使用数据。注意 `ResNetDataset` 的这种设计方式也不是 AutoModel 机制的强制要求，只是设计来帮助更好的实现训练、验证、测试逻辑。开发者也可以自由选择其它方式处理管理训练数据集。

In [ ]:
import os
from torch.utils.data import Dataset
from alphafed.auto_ml.auto_model import ConfigError
from alphafed.auto_ml.cvat.annotation import ImageAnnotationUtils

class ResNetDataset(Dataset):

    def __init__(self,
                 image_dir: str,
                 annotation_file: str,
                 mode: DatasetMode,
                 config: PreprocessorConfig) -> None:
        """Init a dataset instance for ResNet auto model families.

        Args:
            image_dir:
                The directory including image files.
            annotation_file:
                The file including annotation information.
            mode:
                One of training or validation or testing.
            config:
                The configuration for the preprocessor.
        """
        super().__init__()
        if not image_dir or not isinstance(image_dir, str):
            raise ConfigError(f'Invalid image directory: {image_dir}.')
        if not annotation_file or not isinstance(annotation_file, str):
            raise ConfigError(f'Invalid annotation file path: {annotation_file}.')
        assert mode and isinstance(mode, DatasetMode), f'Invalid dataset mode: {mode}.'
        if not os.path.exists(image_dir) or not os.path.isdir(image_dir):
            raise ConfigError(f'{image_dir} does not exist or is not a directory.')
        if not os.path.exists(annotation_file) or not os.path.isfile(annotation_file):
            raise ConfigError(f'{annotation_file} does not exist or is not a file.')

        self.image_dir = image_dir
        self.annotation_file = annotation_file
        # 初始化预处理器
        self.transformer = ResNetPreprocessor(mode=mode, config=config)

        # 加载数据和标注信息
        self.images, self.labels = ImageAnnotationUtils.parse_single_category_annotation(
            annotation_file=self.annotation_file, resource_dir=image_dir, mode=mode
        )

    def __getitem__(self, index: int):
        """读取图片数据时，自动执行预处理器的处理，并提供标注信息。"""
        _item = self.images[index]
        return self.transformer(_item.image_file), _item.class_label

    def __len__(self):
        return len(self.images)

与数据处理相关的工作就完成了，接下来可以设计实现微调逻辑和预测逻辑了。

所有的 AutoModel 实现必须继承 `AutoModel` 基础类。`AutoModel` 基础类定义了一套接口，用以支持自动微调和自动推理两个核心功能。以下为接口定义及其说明：

In [ ]:
from abc import abstractmethod
from typing import Any, Tuple
from alphafed import logger, task_logger

@abstractmethod
def train(self):
    """将 AutoModel 切换到训练模式."""

@abstractmethod
def eval(self):
    """将 AutoModel 切换到推理模式."""

@abstractmethod
def forward(self, *args, **kwargs):
    """配合 __call__ 实现 auto_model(input) 形式的推理调用，此接口需要开发者实现。"""

def __call__(self, *args: Any, **kwargs: Any) -> Any:
    """配合 forward 实现 auto_model(input) 形式的推理调用，此接口不需要开发者实现或修改。"""
    return self.forward(*args, **kwargs)

@abstractmethod
def init_dataset(self, dataset_dir: str) -> Tuple[bool, str]:
    """初始化数据集。

    注意：当前版本数据验证结果是通过判断是否能够成功初始化数据集来判断的，数据验证
    功能将在未来迭代时被单独拆分出来。

    参数说明:
        dataset_dir:
            原始数据（训练、验证、测试）和标注文件的存放目录，由任务管理器调用时告知。
    """

@abstractmethod
def fine_tune(self,
              id: str,
              task_id: str,
              dataset_dir: str,
              is_initiator: bool = False,
              recover: bool = False,
              **kwargs):
    """启动微调训练的接口。

    参数说明:
        id: 当前节点 ID。
        task_id: 当前任务 ID。
        dataset_dir: 原始数据（训练、验证、测试）和标注文件的存放目录，由任务管理器调用时告知。
        is_initiator: 当前节点是否为发起方。
        recover: 是否使用恢复模式启动。
        kwargs: 其它参数，供具体实现扩展。
    """

先来实现 `train` 和 `eval` 接口。AutoModel 训练和推理时，本质上是控制内部的模型内核执行这些操作。AutoModel 是在模型内核之外套了一层壳，所以实现这两个接口主要就是需要切换模型内核的运行模式。
> 代码中用到的一些工具方法与 AutoModel 机制本身无关，开发者可以根据自身喜好自由处理，所以不会详细介绍这些工具方法，只介绍有助于理解 AutoModel 机制的部分。

In [ ]:
import os
import torch
from torch import nn

from alphafed.auto_ml.auto_model import AutoModel, AutoModelError
from res.auto_model_local.res_net import ResNet18

class AutoResNet(AutoModel):

    def _build_model(self):
        """创建并返回一个 ResNet18 模型内核对象。"""
        self._model: nn.Module = ResNet18()
        # self.config 是加载 config.json 后代表配置项的对象，具体实现不影响 AutoModel 机制，此处略过
        self.labels = list(self.config.id2label.values())
        # self._replace_fc_if_diff 是根据实际任务数据的标签列表，更新模型分类层的工具方法
        self._replace_fc_if_diff(len(self.labels))

        # 加载预训练模型初始参数
        param_file = os.path.join(self.resource_dir, self.config.param_file)
        with open(param_file, 'rb') as f:
            state_dict = torch.load(f)
            if self._model.get_parameter('fc.weight').shape[0] != state_dict['fc.weight'].shape[0]:
                raise AutoModelError('The fine tuned labels dismatched the parameters.')
            self._model.load_state_dict(state_dict)

        return self._model.cuda() if self.is_cuda else self._model

    @property
    def model(self) -> nn.Module:
        """返回模型内核对象，本示例中就是返回一个 ResNet18 模型实例。

        为了避免训练过程中错误调用 _build_model 生成新的模型对象，无意间替换了已经经过一定训练的模型
        对象，导致训练结果丢失，通过 @property model 属性将模型对象保护起来。
        """
        if not hasattr(self, '_model'):
            self._model = self._build_model()
        return self._model

    def train(self):
        """将 AutoModel 切换到训练模式."""
        self.model.train()

    def eval(self):
        """将 AutoModel 切换到推理模式."""
        self.model.eval()

接下来实现 `forward` 接口。以使 AutoMode 模型支持通过 auto_model(input) 的方式执行推理操作，以满足部署时接受原始数据类型的要求。`forward` 接口需要配合测试部署使用，所以必须能够处理测试部署时传入的数据格式，本示例中就是需要处理图片文件路径。同时为了方便代码中使用，示例也可以兼顾处理张量形式的输入。开发者也可以根据需要支持更多的格式。

In [ ]:
import json
from typing import overload
from alphafed.auto_ml.auto_model import DatasetMode

from res.auto_model_local.auto import ResNetPreprocessor

class AutoResNet(AutoModel):

    @overload
    def forward(self, input: torch.Tensor) -> str:
        """定义处理输入格式为张量的接口."""

    @overload
    def forward(self, input: str) -> str:
        """定义处理输入格式为图片文件路径的接口."""

    @torch.no_grad()
    def forward(self, input) -> str:
        """实际的实现逻辑。"""
        if not input or not isinstance(input, (str, torch.Tensor)):
            raise AutoModelError(f'Invalid input data: {input}.')
        if isinstance(input, str):
            # 当 input 输入是图片地址时，读入图片，通过预处理器转化为张量。
            if not os.path.isfile(input):
                raise AutoModelError(f'Cannot find or access the image file {input}.')
            # ResNetPreprocessor 是 Preprocessor 的实现，后面介绍
            preprocessor = ResNetPreprocessor(mode=DatasetMode.PREDICTING,
                                              config=self.preprocessor_config)
            input = preprocessor.transform(input)
            input.unsqueeze_(0)
        # 执行推理
        self.model.eval()
        output: torch.Tensor = self.model(input)
        predict = output.argmax(1)[0].item()
        if not self.labels:
            if not os.path.isfile(os.path.join(self.resource_dir, 'fine_tuned.meta')):
                raise AutoModel('The `fine_tuned.meta` file is required to make prediction.')
            with open(os.path.join(self.resource_dir, 'fine_tuned.meta')) as f:
                fine_tuned_json: dict = json.loads(f)
                self.labels = fine_tuned_json.get('labels')
        # 返回类别
        return self.labels[predict]

接下来实现 `init_dataset` 接口。`init_dataset` 加载训练接、数据集、测试集数据，返回创建结果和错误原因（如果失败的话）。

由于历史原因，当前版本 `init_dataset` 接口同时承担了数据验证的职能，预计下一个迭代版本会提供独立的数据验证接口。

In [ ]:
from typing import Optional
from torch.utils.data import DataLoader

class AutoResNet(AutoModel):

    @property
    def training_loader(self) -> DataLoader:
        """返回训练集数据 DataLoader 对象。

        基于与 self.model 同样的理由设计为 @property training_loader
        """
        if not hasattr(self, "_training_loader"):
            self._training_loader = self._build_training_data_loader()
        return self._training_loader

    def _build_training_data_loader(self) -> Optional[DataLoader]:
        """构建训练集数据 DataLoader 对象并返回。"""
        dataset = ResNetDataset(image_dir=self.dataset_dir,
                                annotation_file=self.annotation_file,
                                mode=DatasetMode.TRAINING,
                                config=self.preprocessor_config)
        if len(dataset) == 0:
            return None
        return DataLoader(dataset=dataset,
                          batch_size=self.batch_size,
                          shuffle=True)

    @property
    def validation_loader(self) -> DataLoader:
        """返回验证集数据 DataLoader 对象。

        基于与 self.model 同样的理由设计为 @property validation_loader
        """
        if not hasattr(self, "_validation_loader"):
            self._validation_loader = self._build_validation_data_loader()
        return self._validation_loader

    def _build_validation_data_loader(self) -> DataLoader:
        """构建验证集数据 DataLoader 对象并返回。"""
        dataset = ResNetDataset(image_dir=self.dataset_dir,
                                annotation_file=self.annotation_file,
                                mode=DatasetMode.VALIDATION,
                                config=self.preprocessor_config)
        if len(dataset) == 0:
            return None
        return DataLoader(dataset=dataset, batch_size=self.batch_size)

    @property
    def testing_loader(self) -> DataLoader:
        """返回测试集数据 DataLoader 对象。

        基于与 self.model 同样的理由设计为 @property validation_loader
        """
        if not hasattr(self, "_testing_loader"):
            self._testing_loader = self._build_testing_data_loader()
        return self._testing_loader

    def _build_testing_data_loader(self) -> DataLoader:
        """构建测试集数据 DataLoader 对象并返回。"""
        dataset = ResNetDataset(image_dir=self.dataset_dir,
                                annotation_file=self.annotation_file,
                                mode=DatasetMode.TESTING,
                                config=self.preprocessor_config)
        if len(dataset) == 0:
            return None
        return DataLoader(dataset=dataset, batch_size=self.batch_size)

    def init_dataset(self, dataset_dir: str) -> Tuple[bool, str]:
        """初始化数据集。

        参数说明:
            dataset_dir: 数据及标注文件存放目录。
        """
        self.dataset_dir = dataset_dir
        try:
            if not self._is_dataset_initialized:
                self.training_loader  # 初始化训练集
                self.validation_loader  # 初始化验证集
                self.testing_loader  # 初始化测试集
                if not self.training_loader or not self.testing_loader:
                    logger.error('Both training data and testing data are missing.')
                    return False, 'Must provide train dataset or test dataset to fine tune.'
                self.labels = (self.training_loader.dataset.labels  # 初始化分类标签
                               if self.training_loader
                               else self.testing_loader.dataset.labels)
                self._is_dataset_initialized = True
            return True, 'Initializing dataset complete.'
        except Exception:
            logger.exception('Failed to initialize dataset.')
            return False, '初始化数据失败，请联系模型作者排查原因。'

最后实现 `fine_tune` 接口，实现微调训练逻辑。

In [ ]:
class AutoResNet(AutoModel):

    def fine_tune(self,
                  id: str,
                  task_id: str,
                  dataset_dir: str,
                  is_initiator: bool = False):
        """开始执行微调训练。"""
        self.id = id
        self.task_id = task_id
        self.is_initiator = is_initiator

        # 准备数据
        is_succ, err_msg = self.init_dataset(dataset_dir)
        if not is_succ:
            raise AutoModelError(f'Failed to initialize dataset. {err_msg}')
        # 模型的类别标签默认是和 config.json 中的配置相同的。但是实际使用时，微调所用的数据集标签可能会
        # 发生变化，此时需要根据标注文件中数据集的实际标签更新分类层和标签数据。微调完成后需要更新
        # config.json 文件，保存新的标签配置，推理使用时才能得到正确的结果。
        num_classes = (len(self.training_loader.dataset.labels)
                       if self.training_loader
                       else len(self.testing_loader.dataset.labels))
        self._replace_fc_if_diff(num_classes)

        self.config.id2label = {str(_idx): _label for _idx, _label in enumerate(self.labels)}
        self.config.label2id = {_label: _idx for _idx, _label in enumerate(self.labels)}
        self.config.label2id = dict(sorted(self.config.label2id.items()))

        is_finished = False
        self._epoch = 0
        while not is_finished:
            self._epoch += 1
            self.push_log(f'Begin training of epoch {self._epoch}.')
            # self._train_an_epoch 是执行一个 epoch 训练的逻辑，封装起来只是为了结构清晰，没有特殊之处
            self._train_an_epoch()
            self.push_log(f'Complete training of epoch {self._epoch}.')
            # self._is_finished 是判断训练是否结束的逻辑，可能根据验证集结果提前结束训练，防止过拟合。
            # 封装起来只是为了结构清晰，没有特殊之处
            is_finished = self._is_finished()

        # self._save_fine_tuned 保存微调成果，包括更新 config.json 配置
        self._save_fine_tuned()
        # self._run_test 执行测试，封装起来只是为了结构清晰，没有特殊之处
        avg_loss, correct_rate = self._run_test()
        self.push_log('\n'.join(('Testing result:',
                                 f'avg_loss={avg_loss:.4f}',
                                 f'correct_rate={correct_rate:.2f}')))

`AutoResNet` 的核心功能已经设计完毕，上述代码示例中省去了一些非核心的函数方法和工具，完整的代码在 [res/auto_model_local/auto.py](res/auto_model_local/auto.py) 文件中。

## 调试运行本地模式 AutoModel

接下来的内容展示了如何在模拟环境中调试 `AutoResNet`。由于本地模式的自动建模功能还未在 AlphaMed 平台上线，因此暂时无法在平台上实际运行本地任务，但可以在模拟运行环境中运行测试。

在实际任务处理时，任务管理器会负责为任务准备好两个文件夹，一个用于存放数据和标注，一个用于存放 AutoModel 模型和配置文件。由于模拟环境本身不具备任务管理器的完整功能，所以这个工作需要人工完成。本示例中数据文件夹为 res/data/HAM10000，模型文件夹为 res/auto_model_local。下面开始正式测试，以获取 AutoModel 模型微调和推理的第一手经验。

第一步，先使用 `from_pretrained` 接口加载 AutoModel 模型，看看是否能够成功获得模型对象。由于我们实现的是本地模式的 AutoModel，所以即使不进入模拟环境，在普通的 Notebook 环境下也可以正常运行加载模型、读取数据、执行推理等操作。但是微调训练例外，因为微调训练流程属于 AlphaMed 平台上的一个任务了，需要和任务管理器交互，如果不在模拟环境下运行会出现错误。

In [3]:
# 加载 AutoMode 模型
from alphafed.auto_ml import from_pretrained

auto_model = from_pretrained(resource_dir='res/auto_model_local')

2023-02-07 07:15:45,275|INFO|pretrained|from_pretrained|34:
Loading resouce from: `res/auto_model_local`.
2023-02-07 07:15:45,276|DEBUG|pretrained|_load_model_obj|140:
config.entry_file='auto.py'
2023-02-07 07:15:45,277|DEBUG|pretrained|_load_model_obj|141:
model_file='res/auto_model_local/auto.py'
2023-02-07 07:15:45,353|DEBUG|pretrained|_load_model_obj|143:
module=<module 'auto' from '/app/db/notebook_dir/user_07ec421f72/docs/tutorial/res/auto_model_local/auto.py'>
2023-02-07 07:15:45,354|DEBUG|pretrained|_load_model_obj|145:
model_class=<class 'auto.AutoResNet'>
2023-02-07 07:15:45,355|INFO|pretrained|_load_model_obj|147:
Loading pretrained model complete.


如果第一步模型初始化成功，第二步试试看能否正确加载训练数据。

In [4]:
is_succ, help_text = auto_model.init_dataset(dataset_dir='res/data/HAM10000')
print(f'数据是否加载成功: {is_succ}')
print(f'提示信息: {help_text}')
if is_succ:
    print(f'包含训练集样本: {len(auto_model.training_loader.dataset)}')
    print(f'包含验证集样本: {len(auto_model.validation_loader.dataset)}')
    print(f'包含测试集样本: {len(auto_model.testing_loader.dataset)}')

数据是否加载成功: True
提示信息: Initializing dataset complete.
包含训练集样本: 7012
包含验证集样本: 1000
包含测试集样本: 2003


如果第二步数据加载也成功了，第三步是在微调训练之前做一遍测试，看看初始的预测准确率。记录下来之后可以和微调之后的准确率做个对比，简单评估一下微调的效果。

In [5]:
# _run_test 是示例中实现的测试方法
# 由于本地资源限制，运行时间可能会比较长，需要耐心等待
avg_loss, correct_rate = auto_model._run_test()
print(f'平均损失为: {avg_loss}')
print(f'准确率为: {correct_rate:.2f}')

2023-02-07 07:15:54,181|INFO|auto|push_log|514:
Begin testing of epoch 0.
2023-02-07 07:16:58,864|INFO|auto|_run_test|400:
Testing Average Loss: 2.4185
2023-02-07 07:16:58,865|INFO|auto|_run_test|401:
Testing Correct Rate: 5.59


平均损失为: 2.4184583834392455
准确率为: 5.59


由于本地数据集类别与模型预训练使用的类别完全不同，所以初始准确率很低，只有 5.59%。第四步，使用本地数据执行一轮微调训练。

In [6]:
# 由于本地资源限制，运行时间可能会比较长，需要耐心等待
from alphafed import mock_context

with mock_context(id='ebf4177f-9c4c-4f9b-a895-0409cbbf7338'):
    auto_model.fine_tune(id='ebf4177f-9c4c-4f9b-a895-0409cbbf7338',  # 指定一个假想 ID
                         task_id='3dbcc27e-06de-45ce-aa14-27408b5e8170',  # 指定一个假想 ID
                         dataset_dir='res/data/HAM10000')

2023-02-07 07:17:03,641|WARNING|mock|__enter__|96:
Without specifying nodes, query_nodes returns an empty list.
2023-02-07 07:17:03,650|INFO|auto|push_log|512:
Begin training of epoch 1.
2023-02-07 07:27:09,743|INFO|auto|push_log|512:
Complete training of epoch 1.
2023-02-07 07:27:09,748|INFO|auto|push_log|512:
Begin validation of epoch 1.
2023-02-07 07:27:36,265|INFO|auto|_run_validation|427:
Validation Average Loss: 0.6096
2023-02-07 07:27:36,266|INFO|auto|_run_validation|428:
Validation Correct Rate: 78.80
2023-02-07 07:27:36,267|INFO|auto|push_log|512:
Validation result:
avg_loss=0.6096
correct_rate=78.80
2023-02-07 07:27:36,284|INFO|auto|push_log|512:
Validation result is better than last epoch.
2023-02-07 07:27:36,333|INFO|auto|push_log|512:
Begin testing of epoch 1.
2023-02-07 07:28:28,754|INFO|auto|_run_test|400:
Testing Average Loss: 0.6322
2023-02-07 07:28:28,755|INFO|auto|_run_test|401:
Testing Correct Rate: 77.88
2023-02-07 07:28:28,755|INFO|auto|push_log|512:
Testing resul

从结果上看，即使只经过了一轮微调训练，模型准确率也实现了巨大的提升，从 5.59% 提高到了 77.88%，符合预期。如果想获得更好的效果，可以修改 config.json 文件中的 epochs 配置项，多训练几轮。由于训练比较耗时，暂时先接受目前的训练效果。最后一步，看看如何使用 AutoModel 实现推理。

In [11]:
image_file = 'res/data/HAM10000/ISIC_0025293.jpg'  # 随便选一张图片
predict = auto_model(image_file)
print(f'推理结果为: {predict}')

推理结果为: NV
